In [42]:
import os
import shutil

binaries: raw binaries to run  
dataset: csvs, each subfolder name is corresponding to files in binaries  
output: outputs, each subfolder name is corresponding to files in binaries  

In [43]:
import subprocess

def run_command_with_timeout(command, timeout=600):
    """
    Run a command for a specific duration (timeout) in seconds,
    and capture its output.
    """
    print(command)
    try:
        result = subprocess.run(
            command, 
            shell=True, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE, 
            text=True, 
            timeout=timeout
        )
        return result.stdout + result.stderr
    except subprocess.TimeoutExpired:
        return "Time out after {} seconds.".format(timeout)



In [44]:
run_command_with_timeout("export LD_LIBRARY_PATH=${LD_LIBRARY_PATH:+$LD_LIBRARY_PATH:}`pwd`&&souffle -L /usr/local/src/ddisasm/build/lib/ -j 12 -o ./extracted.bin ./ddisasm.dl")

export LD_LIBRARY_PATH=${LD_LIBRARY_PATH:+$LD_LIBRARY_PATH:}`pwd`&&souffle -L /usr/local/src/ddisasm/build/lib/ -j 12 -o ./extracted.bin ./ddisasm.dl


'Warning: No rules/facts defined for relation functor_data_valid in file ddisasm.dl at line 367\n.decl functor_data_valid(EA:address,Size:unsigned, Res: unsigned)\n------^-----------------------------------------------------------\nWarning: Variable _tmp_53 marked as singleton but occurs more than once in file ddisasm.dl at line 415\n    _tmp_53 = (EA+DisplacementOffset).\n----^----------------------------------\nWarning: Variable _tmp_69 marked as singleton but occurs more than once in file ddisasm.dl at line 422\n    _tmp_69 = as(Size, number).\n----^---------------------------\nWarning: Variable _tmp_70 marked as singleton but occurs more than once in file ddisasm.dl at line 692\n   _tmp_70 = as(Size, number).\n---^---------------------------\nWarning: Variable _tmp_71 marked as singleton but occurs more than once in file ddisasm.dl at line 728\n    _tmp_71 = (8*Size).\n----^-------------------\nWarning: Variable _tmp_72 marked as singleton but occurs more than once in file ddisasm.

In [45]:
run_command_with_timeout("cargo build --release")

cargo build --release


'\x1b\x1b   Compiling\x1b ddisasm-ascent v0.1.0 (/workspaces/ddisasm-ascent)\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: `std::process::Command`\x1b\n\x1b \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/main.rs:2:5\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b\x1b\x1b2\x1b\x1b \x1b\x1b\x1b\x1b|\x1b\x1b \x1b\x1buse std::process::Command;\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\x1b     \x1b\x1b\x1b\x1b^^^^^^^^^^^^^^^^^^^^^\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b  \x1b\x1b\x1b\x1b= \x1b\x1b\x1bnote\x1b\x1b: `#[warn(unused_imports)]` on by default\x1b\n\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: `itertools::Itertools`\x1b\n\x1b \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/main.rs:7:5\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b\x1b\x1b7\x1b\x1b \x1b\x1b\x1b\x1b|\x1b\x1b \x1b\x1buse itertools::Itertools;\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\x1b     \x1b\x1b\x1b\x1b^^^^^^^^^^^^^^^^^^^^\x1b\n\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused variable: `binary_path`\x1b\n\x1b    \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/main.rs:1454:9\x1b\n\x1b     \x1b\

In [46]:
from tqdm import tqdm
for bin in os.listdir("binaries"):
    os.system(f"rm -rf dataset/{bin} output/{bin}")

    os.makedirs(f"output/{bin}", exist_ok=True)
    os.makedirs(f"dataset/{bin}", exist_ok=True)
    os.makedirs(f"output/{bin}/output", exist_ok=True)

    cmd = "LD_LIBRARY_PATH=${LD_LIBRARY_PATH:+$LD_LIBRARY_PATH:}`pwd` "+f"ddisasm --debug-dir dataset/{bin} binaries/{bin}"
    run_command_with_timeout(cmd)


    tmp_dir = os.urandom(8).hex()
    cmd = "LD_LIBRARY_PATH=${LD_LIBRARY_PATH:+$LD_LIBRARY_PATH:}`pwd` "+f"ddisasm -I --debug-dir /tmp/{tmp_dir} binaries/{bin}"
    run_command_with_timeout(cmd)
    shutil.copy(f"/tmp/{tmp_dir}/disassembly/binary.gtirb", f"dataset/{bin}/disassembly/binary.gtirb")


    cmd = "LD_LIBRARY_PATH=${LD_LIBRARY_PATH:+$LD_LIBRARY_PATH:}`pwd` "+f"DDISASM_DEBUG_DIR=dataset/{bin}/disassembly DDISASM_GTIRB_MODULE_NAME={bin} ./extracted.bin -j 16  -F dataset/{bin}/disassembly -D output/{bin}/output"
    output = run_command_with_timeout(cmd)
    with open(f"output/{bin}/ddisasm_output.txt", "w") as f:
        f.write(output)

    ddisasm_out = output.split("\n")
    

    cmd = f"DDISASM_GTIRB_MODULE_NAME={bin} DDISASM_DEBUG_DIR=./dataset/{bin}/disassembly/ LD_LIBRARY_PATH=/usr/local/src/ddisasm/build/lib:$LD_LIBRARY_PATH ./target/release/ddisasm-ascent ./dataset/{bin} binaries/{bin} "
    output = run_command_with_timeout(cmd)
    with open(f"output/{bin}/ascent_output.txt", "w") as f:
        f.write(output)

    for x in tqdm(ddisasm_out):
        if x:
            rel = x.split()[0]
            count  = x.split()[1]
            if f"{rel} size: {count}" not in output:
                print(f"Error in {bin} {rel} size: {count}")


LD_LIBRARY_PATH=${LD_LIBRARY_PATH:+$LD_LIBRARY_PATH:}`pwd` ddisasm --debug-dir dataset/top binaries/top
LD_LIBRARY_PATH=${LD_LIBRARY_PATH:+$LD_LIBRARY_PATH:}`pwd` ddisasm -I --debug-dir /tmp/693761f2acd92052 binaries/top
LD_LIBRARY_PATH=${LD_LIBRARY_PATH:+$LD_LIBRARY_PATH:}`pwd` DDISASM_DEBUG_DIR=dataset/top/disassembly DDISASM_GTIRB_MODULE_NAME=top ./extracted.bin -j 16  -F dataset/top/disassembly -D output/top/output
DDISASM_GTIRB_MODULE_NAME=top DDISASM_DEBUG_DIR=./dataset/top/disassembly/ LD_LIBRARY_PATH=/usr/local/src/ddisasm/build/lib:$LD_LIBRARY_PATH ./target/release/ddisasm-ascent ./dataset/top binaries/top 


100%|██████████| 38/38 [00:00<00:00, 50199.54it/s]
